### **librarys**

In [ ]:
import numpy as np
import random
import math
from array import *
import copy

### **Tasks Dataset**

In [ ]:
def datatask(NT):
    
    Task_arr=[]
    Task_sort=[]
    #random.seed(400)
    for i in range(NT):
        ##First Type
        T_rand=random.randrange(NT)%2
        task={'mem' : None, 'size': None,'dead': None}
        #print('random',random)
        if(T_rand==0):  
            task['size']= random.randrange(1028,4280)
            task['dead']= random.randrange(500,2500)/1000
            task['mem']= random.randrange(64,1024)
        ##Second Type    
        if(T_rand==1):
            task['size']= random.randrange(5123,9784)
            task['dead']= random.randrange(2500,10000)/1000
            task['mem']= random.randrange(64,1024)
            

        Task_arr.append(task) 
        Task_sort.append(task) 
    #print(Task_arr)      
        
    return Task_arr,Task_sort

### **Nodes Dataset**

In [ ]:
def datanode(NF,NC):
    #random.seed(400)
    vmF_arr = []
    vmC_arr = []
    N_VMF=np.zeros(NF)
    N_VMC=np.zeros(NC)
    for j in range(NC):
        N_VMC[j]= random.randrange(4,9)
    #random.seed(400)
    for i in range(NF):
        N_VMF[i]= random.randrange(1,5)
    
    # print(N_VMF)
    # print(N_VMC)
    all_vmF=int(sum(N_VMF))
    all_vmC=int(sum(N_VMC))
    #print(all_vmF)
    #print(all_vmC)
    all_vm=(all_vmF+all_vmC) 
    #print("all_virtual_machines",all_vm)
    
    ##Specification of cloud and fog nodes
    for i in range(len(N_VMC)):
        dell = 150/1000
        pb = 50
        for j in range(int(N_VMC[i])):
            vm = {'mem' : None, 'cpu': None,'p_i': None, 'p_a': None, 'p_b': None, 'del': None}
            vm['mem'] = random.randrange(1024,4096)
            vm['cpu'] = random.randrange(4000,8000)
            vm['p_a'] = random.randrange(50,200)
            vm['p_i'] = (random.randrange(60,70)*vm['p_a'] )/100
            vm['p_b'] = pb
            vm['del'] = dell
            vmC_arr.append(vm)
    #random.seed(400)        
    for i in range(len(N_VMF)):
        dell = random.randrange(2,10)/1000
        pb = 20
        for j in range(int(N_VMF[i])):
            vm = {'mem' : None, 'cpu': None,'p_i': None, 'p_a': None, 'p_b': None, 'del': None}
            vm['mem'] = random.randrange(1024,2048)
            vm['cpu'] = random.randrange(500,4000)
            vm['p_a'] = random.randrange(20,50)
            vm['p_i'] = (random.randrange(60,70)*vm['p_a'] )/100
            vm['p_b'] = pb
            vm['del'] = dell
            vmF_arr.append(vm)
        
    
            
    VM=vmF_arr+vmC_arr
    return vmF_arr, vmC_arr,VM, all_vm,all_vmF,all_vmC 

### **Sort Task by Deadline**

In [ ]:
def sort_deadline(Task_sort):
    new_Task_sort=copy.deepcopy(Task_sort)
    
    for i in range(Num_T-1):
        for j in range(0, Num_T-i-1):
             if new_Task_sort[j]['dead'] > new_Task_sort[j+1]['dead'] : 
                new_Task_sort[j]['dead'], new_Task_sort[j+1]['dead'] = new_Task_sort[j+1]['dead'], new_Task_sort[j]['dead']
                new_Task_sort[j]['size'], new_Task_sort[j+1]['size'] = new_Task_sort[j+1]['size'], new_Task_sort[j]['size']
                new_Task_sort[j]['mem'], new_Task_sort[j+1]['mem'] = new_Task_sort[j+1]['mem'], new_Task_sort[j]['mem']
    #print(new_Task_sort)
    return new_Task_sort

### **Memory Condition**

In [ ]:
def Memory_condition():
    locate=np.zeros((Num_T,all_vm))
    for i in range(Num_T):
        for j in range(all_vm):
            if (new_Task_sort[i]['mem']<=VM[j]['mem']):
                locate[i,j]=1
    #print("locate",locate)
    return locate

## **Algorithms**

### **EDF Algorithm**

In [ ]:
def EDf():
    #print("@@@@@@@@@@@@@@@@  EDF Algorithm @@@@@@@@@@@@@")
    Sol_EDF=np.zeros((Num_T))
    locate=Memory_condition()
    #print(mlocate)
##################choice random ###################
    indices=[]
    
    for i in range (locate.shape[0]):
        row=np.array(locate[i, :])
        indices_row=np.argwhere(row==1)
        #print(indices_row) 
        indices_row=random.sample(indices_row.tolist(),1)
        indices.append(indices_row)

     
    #print(indices) 
    for j in range(Num_T):
        Sol_EDF[j]=int(np.array(indices[j]))
        #print("Sol_EDF",j,":",Sol_EDF[j])
    #print("Sol_EDF",":",Sol_EDF)            

    mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART=calculate_Results(Sol_EDF)        
                
                    
    return mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART
                
        

### **Calculate Results**

In [ ]:
def calculate_Results(solution):
    res=np.zeros((Num_T,all_vm),dtype=np.float_)
    exe=np.zeros((Num_T,all_vm),dtype=np.float_)
    avail=np.zeros((all_vm),dtype=np.float_)
    Pnalti=0
    number_MD=0
    RT=0
    ART=0
    Pnalti_s1=0
    Pnalti_s2=0
    Pnalti_s3=0
    number_MD_s1=0
    number_MD_s2=0
    number_MD_s3=0
    x1=0
    x2=0
    x3=0
    energy_VM=np.zeros((all_vm),dtype=np.float_)
    energy_N=np.zeros((Num_FN+Num_CN),dtype=np.float_)
    ## calculate response time
    for i in range(Num_T):
        N=int(solution[i])
        exe[i,N]=new_Task_sort[i]['size']/VM[N]['cpu']
        #print("exe",i,",",N,":",exe[i,N])
        res[i,N]=exe[i,N]+avail[N]+2*VM[N]['del']
        avail[N]+=exe[i,N]
        #print("avail",N,":",avail[N])
        RT+=res[i,N]
        ## deadline condition
        if(res[i,N]>new_Task_sort[i]['dead']):
            #print(i,node)
            #print(resTN[i,node],D[i])
            Pnalti+=(res[i,N]-new_Task_sort[i]['dead'])
            if(new_Task_sort[i]['size']<4280):
                x1+=1
                Pnalti_s1+=(res[i,N]-new_Task_sort[i]['dead'])
#             if((new_Task_sort[i]['size']>=1028) and (new_Task_sort[i]['size']<=4280)):
#                 x2+=1
#                 Pnalti_s2+=(res[i,N]-new_Task_sort[i]['dead'])
            if(new_Task_sort[i]['size']>=4280):
                x2+=1
                Pnalti_s2+=(res[i,N]-new_Task_sort[i]['dead'])
            
        if(res[i,N]<=new_Task_sort[i]['dead']):
            number_MD+=1
            if(new_Task_sort[i]['size']<4280):
                number_MD_s1+=1
                x1+=1
#             if((new_Task_sort[i]['size']>=1028) and (new_Task_sort[i]['size']<=4280)):
#                 number_MD_s2+=1
#                 x2+=1
            if(new_Task_sort[i]['size']>=4280):
                number_MD_s2+=1
                x2+=1
    DSR=(number_MD*100)/Num_T
    DSR1=(number_MD_s1*100)/x1
    DSR2=(number_MD_s2*100)/x2
#     DSR3=(number_MD_s3*100)/x3
    ART=RT/Num_T
#     print('ARt',ART)
#     print('Pnalti_s1',Pnalti_s1)
#     print('Pnalti_s2',Pnalti_s2)
# #     print('Pnalti_s3',Pnalti_s3)
#     print('DSR1',DSR1)
#     print('DSR2',DSR2)
# #     print('DSR3',DSR3)
#     print('Pnalti',Pnalti)
#     print('number_MD',number_MD)
#     print('DSR',DSR)
#     print('avail',avail)
    mak=np.max(avail)
#     print('makespan:',mak)
    for j in range(all_vm):
        energy_VM[j]=((VM[j]['p_a']*(avail[j])) +(mak-(avail[j]))*VM[j]['p_i'])
        #print('energy',j,':',energy[j])
    for k in range(Num_FN+ Num_CN):
        if (k<Num_FN):
            energy_N[k]=20*mak
        if (Num_FN<=k): 
            energy_N[k]=50*mak 
    total_energy_VM=sum(energy_VM)
    total_energy_N=sum(energy_N)
    T_energy=(total_energy_VM+total_energy_N)
#     print("total_energy_VM",total_energy_VM)
#     print("total_energy_N",total_energy_N)
#     print("T_energy",T_energy)
    
    return mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART

### **SFC Algorithm**

In [ ]:
def SFC():
    #print("@@@@@@@@@@@@@@@@  SFC Algorithm @@@@@@@@@@@@@")
    resF=np.zeros((Num_T,all_vmF),dtype=np.float_)
    resC=np.zeros((Num_T,all_vmC),dtype=np.float_)
    exe=np.zeros((Num_T,all_vm),dtype=np.float_)
    avail=np.zeros((all_vm),dtype=np.float_)
    Sol_SFC=np.zeros((Num_T))
    
    Flocate=np.zeros((all_vmF))
    Clocate=np.zeros((all_vmC))
    
            
    

    for i in range(Num_T):
        if (new_Task_sort[i]['dead']<=(2500/1000)):
            #print("type1")
            for j in range(all_vmF):
                if (new_Task_sort[i]['mem']<=VM[j]['mem']):
                    Flocate[j]=1
                exe[i,j]=new_Task_sort[i]['size']/VM[j]['cpu']
                resF[i,j]=exe[i,j]+avail[j]+(2*VM[j]['del'])
            #print("Flocate",Flocate)
            #print("exe",exe)
            #print("resF",resF)
            index_min_res=np.array(np.where(resF[i,:]==(np.min(resF[i,np.where(Flocate[:]==1)])))) 
            #print("index_min_res",index_min_res)
            node=int(np.min(index_min_res))
            Sol_SFC[i]=node
            avail[node]+=new_Task_sort[i]['size']/VM[node]['cpu']
            
           
        if (new_Task_sort[i]['dead']>(2500/1000)):
            #print("type2")
            R=all_vmF
            for j in range(all_vmC):
                if (new_Task_sort[i]['mem']<=VM[j+R]['mem']):
                    Clocate[j]=1
                exe[i,j+R]=new_Task_sort[i]['size']/VM[j+R]['cpu']
                resC[i,j]=exe[i,j+R]+avail[j+R]+(2*VM[j+R]['del'])
            #print("Clocate",Clocate)
            #print("exe",exe)
            #print("resC",resC)
            index_min_res=np.array(np.where(resC[i,:]==(np.min(resC[i,np.where(Clocate[:]==1)])) )) 
            #print("index_min_res",index_min_res)
            node=int(np.min(index_min_res))
            Sol_SFC[i]=node+R
            avail[node+R]+=new_Task_sort[i]['size']/VM[node+R]['cpu']
        #print("avail",avail)
#     print("Sol_SFC",Sol_SFC)
    
    mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART=calculate_Results(Sol_SFC)   
    return mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART        

### **Grasp Algorithm**

In [ ]:
def GRASP(Pre):
    #print("@@@@@@@@@@@@@@@@  GRASP Algorithm @@@@@@@@@@@@@")
    
    MClocate=np.zeros((Num_T,all_vm))
    Sol_SG=np.zeros((Num_T))
    Exe=np.zeros((Num_T,all_vm),dtype=np.float_)
    Res=np.zeros((Num_T,all_vm),dtype=np.float_)
    avail=np.zeros((all_vm),dtype=np.float_)
    for i in range(Num_T):
        avail,Sol_SG,Exe,Res=Compare_Res(i,avail,Sol_SG,Exe,Res)
        #print("Sol_SG",Sol_SG)
        
    #####LOCAL SEARCH#######
    
    Sol_C=np.empty_like (Sol_SG)
    Sol_C[:]=Sol_SG
    Sol_new=np.empty_like (Sol_C)    
    TN=int(Num_T-1)
    for i in range(TN):
        for j in range(i+1,Num_T):
            #print(Sol_C[:])
            Sol_new[:]=Sol_C
            node1=int(Sol_C[i])
            node2=int(Sol_C[j])
            #print("node1",node1)
            #print("node2",node2)
            Sol_new[i]=node2
            Sol_new[j]=node1
            #print("Sol_new",Sol_new[:])
            DSR_C,T_energy_C=Calculate_DSR_E(Sol_C)
            DSR_N,T_energy_N=Calculate_DSR_E(Sol_new)
#             print(DSR_C,T_energy_C)
#             print(DSR_N,T_energy_N)
            if(DSR_C<DSR_N):
                Sol_C[:]=Sol_new[:]
                #print("Sol_C[:]",Sol_C[:])
                
            if((DSR_C==DSR_N) and(T_energy_N<T_energy_C) ):
                Sol_C[:]=Sol_new[:]
                #print("Sol_C[:]",Sol_C[:])
                

    
    
    
    
    
    
    
    
                
    #print("Sol_C",Sol_C)       
            
    mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART=calculate_Results(Sol_C)  
    return mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART
    
    
    

In [ ]:
def Compare_Res(i,avail,Sol_SG,Exe,Res):
    task=i
    avail_node=avail
    M_locate=np.zeros((Num_T,all_vm),dtype=np.float_)
    
    for j in range(all_vm):
        if(new_Task_sort[task]['mem']<=VM[j]['mem']):
            M_locate[task,j]=1
            Exe[task,j]=new_Task_sort[task]['size']/VM[j]['cpu']
            #print(exeTN[task,j])
            Res[task,j]=Exe[task,j]+avail_node[j]+(2*VM[j]['del'])
            #print(resTN[task,j])
        if(new_Task_sort[task]['mem']>VM[j]['mem']):
            Exe[task,j]=1000000000
            #print(exeTN[task,j])
            Res[task,j]=Exe[task,j]+avail_node[j]+(2*VM[j]['del'])
            #print(resTN[task,j])
    row=np.array(M_locate[task,:])
    indices_row=np.argwhere(row==1)
    if(len(indices_row)==1):
        node=int(np.array(indices_row))
        Sol_SG[task]=node
        avail_node[node]+=new_Task_sort[task]['size']/VM[node]['cpu']
    if(len(indices_row)>=2):
        if(Pre==0):
            index_min_res=np.array(np.where(Res[task,:]==np.min(Res[task,np.where(row[:]==1)])))
            #print('index_min_res',index_min_res)
            node=int(np.min(index_min_res))
            Sol_SG[task]=node
            avail_node[node]+=new_Task_sort[task]['size']/VM[node]['cpu']
        if(Pre==100):
            indices_row=random.sample(indices_row.tolist(),1)
            #print('indices_row',indices_row)
            node=int(np.array(indices_row))
            Sol_SG[task]=node
            avail_node[node]+=new_Task_sort[task]['size']/VM[node]['cpu']
        
        if(Pre>1 and Pre!=100):
            new_node= np.arange(all_vm)
            new_resTN=np.empty_like (Res)
            new_resTN[:,:]=Res
            #print(new_resTN)
            for i in range(all_vm-1):
                for j in range(0, all_vm-i-1):
                    if new_resTN[task,j] > new_resTN[task,j+1] : 
                        new_resTN[task,j] , new_resTN[task,j+1] = new_resTN[task,j+1], new_resTN[task,j]
                        new_node[j],new_node[j+1]=new_node[j+1],new_node[j]
            #print(new_resTN)
            #print(new_node)
            pre_random=math.ceil((Pre*len(indices_row))/100)
            #print('pre_random',pre_random)
            random_node=np.random.randint(0,pre_random)
            #print('random_node',random_node)
            node=int(new_node[random_node])
            Sol_SG[task]=node
            avail_node[node]+=new_Task_sort[task]['size']/VM[node]['cpu']
        return avail_node,Sol_SG,Exe,Res

In [ ]:
def Calculate_DSR_E(solution):
    res=np.zeros((Num_T,all_vm),dtype=np.float_)
    exe=np.zeros((Num_T,all_vm),dtype=np.float_)
    avail=np.zeros((all_vm),dtype=np.float_)
    Pnalti=0
    number_MD=0
    RT=0
    ART=0
    Pnalti_s1=0
    Pnalti_s2=0
    Pnalti_s3=0
    number_MD_s1=0
    number_MD_s2=0
    number_MD_s3=0
    x1=0
    x2=0
    x3=0
    energy_VM=np.zeros((all_vm),dtype=np.float_)
    energy_N=np.zeros((Num_FN+Num_CN),dtype=np.float_)

    for i in range(Num_T):
        N=int(solution[i])
        #print("N",N)
        exe[i,N]=new_Task_sort[i]['size']/VM[N]['cpu']
#         print("exe[i,N]",i,N,exe[i,N])
        res[i,N]=exe[i,N]+avail[N]+2*VM[N]['del']
        avail[N]+=exe[i,N]
        RT+=res[i,N]
        
        if(res[i,N]>new_Task_sort[i]['dead']):
            #print(i,node)
            #print(resTN[i,node],D[i])
            Pnalti+=(res[i,N]-new_Task_sort[i]['dead'])
            if(new_Task_sort[i]['size']<4280):
                x1+=1
                Pnalti_s1+=(res[i,N]-new_Task_sort[i]['dead'])
#             if((new_Task_sort[i]['size']>=1028) and (new_Task_sort[i]['size']<=4280)):
#                 x2+=1
#                 Pnalti_s2+=(res[i,N]-new_Task_sort[i]['dead'])
            if(new_Task_sort[i]['size']>=4280):
                x2+=1
                Pnalti_s2+=(res[i,N]-new_Task_sort[i]['dead'])
            
        if(res[i,N]<=new_Task_sort[i]['dead']):
            number_MD+=1
            if(new_Task_sort[i]['size']<4280):
                number_MD_s1+=1
                x1+=1
#             if((new_Task_sort[i]['size']>=1028) and (new_Task_sort[i]['size']<=4280)):
#                 number_MD_s2+=1
#                 x2+=1
            if(new_Task_sort[i]['size']>=4280):
                number_MD_s2+=1
                x2+=1
    DSR=(number_MD*100)/Num_T
    DSR1=(number_MD_s1*100)/x1
    DSR2=(number_MD_s2*100)/x2
#     DSR3=(number_MD_s3*100)/x3
    ART=RT/Num_T
#     print('ARt',ART)
#     print('Pnalti_s1',Pnalti_s1)
#     print('Pnalti_s2',Pnalti_s2)
#     print('Pnalti_s3',Pnalti_s3)
#     print('DSR1',DSR1)
#     print('DSR2',DSR2)
#     print('DSR3',DSR3)
#     print('Pnalti',Pnalti)
#     print('number_MD',number_MD)
#     print('DSR',DSR)
#     print('avail',avail)
    mak=np.max(avail)
#     print('makespan:',mak)
    for j in range(all_vm):
        energy_VM[j]=((VM[j]['p_a']*(avail[j])) +(mak-(avail[j]))*VM[j]['p_i'])
        #print('energy',j,':',energy[j])
    for k in range(Num_FN+ Num_CN):
        if (k<Num_FN):
            energy_N[k]=20*mak
        if (Num_FN<=k): 
            energy_N[k]=50*mak 
    total_energy_VM=sum(energy_VM)
    total_energy_N=sum(energy_N)
    T_energy=(total_energy_VM+total_energy_N)
#     print("total_energy_VM",total_energy_VM)
#     print("total_energy_N",total_energy_N)
#     print("T_energy",T_energy)
    return DSR,T_energy

### **HHOLS Algorithm**

**Generate population**

In [ ]:
def generate_population():
    pop=np.zeros((N_Pop,Num_T))
    sol=np.zeros((N_Pop,Num_T))
    locate=np.zeros((Num_T,all_vm))
    for i in range(Num_T):
        for j in range(all_vm):
            if (Task_arr[i]['mem']<=VM[j]['mem']):
                locate[i,j]=1
#     print("locate",locate)
#     print("locate.shape[0]",locate.shape[0])
##################choice random ###################
    indices=[]
    for k in range(N_Pop):
        indices[:] = []
        for i in range (Num_T):
            row=np.array(locate[i,:])
            indices_row=np.argwhere(row==1)
            indices_row=random.sample(indices_row.tolist(),1)
            indices.append(indices_row)
            #print("indices",indices)
            pop[k,i]= int(np.array(indices[i]))+1
    
  
        

            
    #print (pop)  
    
    return pop

**Calculate Fitness for** **Population**

In [ ]:
def calculate_fitness(population):
    #print('***************************calculate_fitness***********************')
    #population=generate_population()
    avail=np.zeros((N_Pop,all_vm),dtype=np.float_)
    
    fitness=np.zeros((N_Pop),dtype=np.float_)
    total_energy=np.zeros((N_Pop),dtype=np.float_)
    total_energy_N=np.zeros((N_Pop),dtype=np.float_)
    t_energy=np.zeros((N_Pop),dtype=np.float_)
    energy=np.zeros((N_Pop,all_vm),dtype=np.float_)
    energy_N=np.zeros((N_Pop,Num_FN+Num_CN),dtype=np.float_)
    makespan=np.zeros((N_Pop),dtype=np.float_)
    A=0.8
    
    for i in range(N_Pop):
        #print(population[i,:])
        for j in range(Num_T):
            node=int(population[i,j])-1
            #print(node)
            e=Task_arr[j]['size']/VM[node]['cpu']
            #print('e',e)
            avail[i,node]=avail[i,node]+e
            #print(avail[i,:])  
    for i in range(N_Pop):
        makespan[i]=max(avail[i])
        #print('makespan',makespan[i])
        for j in range(Num_T):
            node=int(population[i,j])-1
            #print('node',node)
            
            energy[i,node]+=((VM[node]['p_a']*(avail[i,node]))+((makespan[i]-(avail[i,node]))*VM[node]['p_i']))
        for k in range(Num_FN+ Num_CN):
            if (k<Num_FN):
                energy_N[i,k]=20*makespan[i]
            if (Num_FN<=k): 
                energy_N[i,k]=50*makespan[i]
        total_energy_N[i]=sum(energy_N[i,:])
        total_energy[i]=sum(energy[i,:])
        t_energy[i]=(total_energy_N[i]+total_energy[i])
    
        fitness[i]=(A*t_energy[i])+((1-A)*makespan[i])
#     print('makespan',makespan)
#     print('energy',energy)
#     print(' total_energy', t_energy)
#     print('fitness',fitness) 
    best_fitness=int(np.min(np.array(np.where(fitness==np.min(fitness[:])))))
#     print(best_fitness)
    best_position=population[best_fitness,:]
#     print(best_position)
    return best_position



**Update initial** **Energy**

In [ ]:
def update_E(z):
    #print('*****************update_E*****************')
    E0=random.uniform(-1,1)
    #print('E0',E0)
    E=2*E0*(1-(z/max_iter))
    return E

**Calculate Fitness for compare Solution**

In [ ]:
def calculate_F(a):
    #print('******************calculate_F******************')
    lenght=len(a)
    avail=np.zeros((all_vm),dtype=np.float_)
    energy_VM=np.zeros((all_vm),dtype=np.float_)
    energy_N=np.zeros((Num_FN+Num_CN),dtype=np.float_)
    A=0.8
    fitness=0
    total_energy=0
    for i in range(Num_T):
        node=int(a[i])-1
        e=Task_arr[i]['size']/VM[node]['cpu']
        avail[node]=avail[node]+e
    makespan=max(avail)
    #print('makespan',makespan)
    for j in range(all_vm):
        #print('avail[j]',avail[j])
        #print('fbeta[node],falpha[node],avail[i,node]',fbeta[j],falpha[j],avail[j])
        energy_VM[j]=((VM[j]['p_a']*(avail[j])) +((makespan-(avail[j]))*VM[j]['p_i']))
    #print('energy',energy)
    for k in range(Num_FN+ Num_CN):
        if (k<Num_FN):
            energy_N[k]=20*makespan
        if (Num_FN<=k): 
            energy_N[k]=50*makespan
    total_energy_VM=sum(energy_VM)
    total_energy_N=sum(energy_N)
    total_energy=(total_energy_VM+total_energy_N)
    
    #print('total_energy',total_energy)
    fitness=(A*total_energy)+((1-A)*makespan)
    return fitness

**Normalize**

In [ ]:
def calculate_FAndNormalize(a):
    #print('******************calculate_FAndNormalize******************')
    lenght=len(a)
    a=np.array(a)
    #minimum=1
    #maximum=n
    minimum=np.min(a)
    maximum=np.max(a)
    A=0.8
    #print('min,max',minimum,maximum)
    normalize=(a-minimum)/(maximum-minimum)
    #print('normalize',normalize)
    scale=np.round(normalize*(all_vm-1)+1)
    b=np.array(scale)
    #print('scale',scale)
    avail=np.zeros((all_vm),dtype=np.float_)
    energy_VM=np.zeros((all_vm),dtype=np.float_)
    energy_N=np.zeros((Num_FN+Num_CN),dtype=np.float_)
    for i in range(Num_T):
        node=int(b[i])-1
        #print(node)
        e=Task_arr[i]['size']/VM[node]['cpu']
        avail[node]=avail[node]+e
    makespan=max(avail)
    for j in range(all_vm):
        energy_VM[j]=((VM[j]['p_a']*(avail[j])) +((makespan-(avail[j]))*VM[j]['p_i']))
    total_energy_VM=np.sum(energy_VM)
    for k in range(Num_FN+ Num_CN):
        if (k<Num_FN):
            energy_N[k]=20*makespan
        if (Num_FN<=k): 
            energy_N[k]=50*makespan
    total_energy_N=sum(energy_N)
    total_energy=(total_energy_VM+total_energy_N)
    fitness=(A*total_energy)+((1-A)*makespan)
    #print('fitness',fitness)
    return fitness

**Local Search Strategy for HHO**

In [ ]:
def local_search_strategy(y_rabbit):
    #print('***************************local_search_strategy***********************')
    
    y_improved=np.empty_like (y_rabbit)
    y_improved[:]=y_rabbit
    #print('y_improved',y_improved)
    L=len(y_improved[:])
    total_exe=np.zeros((all_vm),dtype=np.float_)
    for i in range(L):
        node=int(y_improved[i])-1
        #print(node)
        e=Task_arr[i]['size']/VM[node]['cpu']
        #print('e',e)
        total_exe[node]=total_exe[node]+e
    #print('total_exe',total_exe)
    makespan=np.max(total_exe)
    VM_makespan=np.min(np.where(total_exe==np.max(total_exe[:])))+1
    min_exe=np.min(total_exe)
    VM_min=np.min(np.where(total_exe==np.min(total_exe[:])))+1
    #print('makespan,min_exe',makespan,min_exe,VM_makespan,VM_min)
    task_VM=np.argwhere(y_improved==VM_makespan)
    #print('task on VM_makespan',task_VM)
    task_random=random.sample(task_VM.tolist(),1)
    random_task=int(np.array(task_random))
    #print(task_random,random_task)
    y_improved[random_task]=VM_min
    #print('y_improved',y_improved)
    #print('y_rabbit',y_rabbit)
    f_y_improved=calculate_F(y_improved)
    f_y_rabbit=calculate_F(y_rabbit)
    #print('f_y_improved,f_y_rabbit',f_y_improved,f_y_rabbit)
    if(f_y_improved<f_y_rabbit):
        y_rabbit[:]=y_improved
    return y_rabbit

**Main HHOLS Algorithm**

In [ ]:
def algorithm3():
    #print('@@@@@@@@@@@@@@@  HHOLS Algorithm   @@@@@@@@@@@')
    population=generate_population()
#     print(population)
    y_rabbit=calculate_fitness(population)
#     print(y_rabbit)
    y_mean=np.zeros((Num_T),dtype=np.float_)
    y=np.zeros((max_iter+2,Num_T),dtype=np.float_)
    normalize=np.zeros((max_iter+2,Num_T),dtype=np.float_)
    scale=np.zeros((max_iter+2,Num_T),dtype=np.float_)
    #print(y)
    z=1
    
    random_y=np.random.randint(0,N_Pop, size=(1))
    #y[1]=np.array(population[random_y,:])
    y[1]= y_rabbit
    #print('y[1]',y[1])
    while (z<=max_iter):
        #print('y_rabbit',y_rabbit)
        E=update_E(z) 
        #print('E',E)
        if(abs(E)>=1):
            p=np.random.random()
            #print('p,E',p,E)
            #print('p',p)
            if(p>=0.5):
                r1=np.random.random()
                r2=np.random.random()
                random_y=np.random.randint(0,N_Pop, size=(1))
                #print('random_y',random_y)
                y_r=np.zeros((Num_T),dtype=np.float_)
                y_r=np.array(population[random_y,:])
                #print(y_r)
                y[z+1,:]=y_r-(r1*(abs(y_r-(2*r2*y[z]))))
                #print('y[z+1] , eq2',y[z+1,:])
            if(p<0.5):
              
                r3=np.random.random()
                r4=np.random.random()
                L=1
                U=all_vm
                fy_mean=(np.sum(population,axis=0))/N_Pop
                
#                 for i in range(N_Pop):
#                     #print('population',population[i,:])
#                     y_mean+=population[i,:]
#                     #print('y_mean',y_mean)
#                     fy_mean=y_mean/N_Pop
#                     #print('fy_mean',fy_mean)
                    
                y[z+1,:]=y_rabbit-fy_mean-(r3*(L+(r4*(U-L))))
                #print('y[z+1] , eq2',y[z+1,:])
        if(abs(E)<1):
            p=np.random.random()
            #print('p,E',p,E)
            
            if((p>=0.5) and (abs(E)>=0.5)):
                dy=abs(y_rabbit-y[z])
                rand=np.random.random()
                J=2*(1-rand)
                y[z+1,:]=dy-E*abs(J*y_rabbit-y[z])
                #print('y[z+1] in soft besiege',y[z+1,:])
                
            if((p>=0.5) and (abs(E)<0.5)):
                dy=abs(y_rabbit-y[z])
                y[z+1,:]=y_rabbit-E*(abs(dy))
                #print('y[z+1] in hard besiege',y[z+1,:])
                
            if((p<0.5) and (abs(E)>=0.5)):
                u=np.random.random()
                v=np.random.random()
                B=1.5
                sigma=((math.gamma(1+B)* math.sin( math.pi/2))/(math.gamma((1+B)/2)*B*(2**((B-1)/2))))**(1/B)
                levy=0.01*((u*sigma)/(abs(v)**(1/B)))
                #print('levy',levy)
                rand=np.random.random()
                J=2*(1-rand)
                K= y_rabbit-abs(E*(abs(J*y_rabbit-y[z])))
                #print('k',K)
                random_S=np.random.randint(0,N_Pop, size=(1))
                s=np.zeros((Num_T ),dtype=np.float_)
                s[:]=population[random_S,:]
                #print('s',s)
                Z=K+s*levy
                #print('Z',Z)
                f_y=calculate_FAndNormalize(y[z])
                f_k=calculate_FAndNormalize(K)
                f_z=calculate_FAndNormalize(Z)
                
                if((f_k < f_y) and (f_k<f_z)):
                    y[z+1]=K
                if(f_z <f_y):
                    y[z+1,:]=Z
                else:
                    y[z+1]=y[z]
                
                #print('y[z+1] in soft besiege PRD',y[z+1,:])
            if((p<0.5) and (abs(E)<0.5)):
                
                fy_mean=(np.sum(population,axis=0))/N_Pop
#                 for i in range(N_Pop):
#                     #print('population',population[i,:])
#                     y_mean+=population[i,:]
#                     #print('y_mean',y_mean)
#                     fy_mean=y_mean/N_Pop
                #print('fy_mean',fy_mean)
                rand=np.random.random()
                J=2*(1-rand)
                y[z+1,:]=y_rabbit-E*(abs(J*y_rabbit-fy_mean))
                #print('y[t+1] in hard besiege PRD]',y[z+1,:])
        
        minimum=np.min(y[z+1,:])
        maximum=np.max(y[z+1,:])
        #minimum=1
        #maximum=n
        #print('max ,min',minimum,maximum)
        normalize[z+1,:]=(y[z+1,:]-minimum)/(maximum-minimum)
        #print('normalize',normalize)
        scale[z+1,:]=np.round(normalize[z+1,:]*(all_vm-1)+1)
        y[z+1,:]=scale[z+1,:]
        #print('scale',scale[z+1,:])
        #print('y[',z+1,']',y[z+1,:])
        random=np.array(np.random.randint(0,Num_T,size=(2)))
        rand1=random[0]
        rand2=random[1]
        node=np.array(y[z+1,random])
        node1=node[0]
        node2=node[1]
        #print('random,node',random,node)
        #print('rand1,rand2',rand1,rand2)
        #print('node1,node2',node1,node2)
        swap=np.empty_like (y[z+1,:])
        swap[:]=y[z+1,:]
        swap[rand1]=node2
        swap[rand2]=node1
        #print('swap',swap)
        F_fy=calculate_F(y_rabbit)
        F_swap=calculate_F(swap)
        #print('fitness',F_fy,F_swap)
        if(F_swap<F_fy):
            y_rabbit=swap[:]
        #print('y_rabbit',y_rabbit)
        y_rabbit=local_search_strategy(y_rabbit)   
        #print('y_rabbit',y_rabbit)  
        z +=1
    final_y_rabbit=y_rabbit
#     print('final_y_rabbit',final_y_rabbit)
    
    mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART=calculate_result_H(final_y_rabbit)
    
    return mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART

**Calculate Result HHOLS**

In [ ]:
def calculate_result_H(solution):
    ###########################calculate_responseTime
    res=np.zeros((Num_T,all_vm),dtype=np.float_)
    exe=np.zeros((Num_T,all_vm),dtype=np.float_)
    avail=np.zeros((all_vm),dtype=np.float_)
    Pnalti=0
    number_MD=0
    RT=0
    ART=0
    Pnalti_s1=0
    Pnalti_s2=0
    Pnalti_s3=0
    number_MD_s1=0
    number_MD_s2=0
    number_MD_s3=0
    x1=0
    x2=0
    x3=0
    energy_VM=np.zeros((all_vm),dtype=np.float_)
    energy_N=np.zeros((Num_FN+Num_CN),dtype=np.float_)

    for i in range(Num_T):
        N=int(solution[i])-1
        exe[i,N]=Task_arr[i]['size']/VM[N]['cpu']
        res[i,N]=exe[i,N]+avail[N]+2*VM[N]['del']
        avail[N]+=exe[i,N]
        RT+=res[i,N]
        if(res[i,N]>Task_arr[i]['dead']):
            #print(i,node)
            #print(resTN[i,node],D[i])
            Pnalti+=(res[i,N]-Task_arr[i]['dead'])
            if(Task_arr[i]['size']<4280):
                x1+=1
                Pnalti_s1+=(res[i,N]-Task_arr[i]['dead'])
#             if((Task_arr[i]['size']>=1028) and (Task_arr[i]['size']<=4280)):
#                 x2+=1
#                 Pnalti_s2+=(res[i,N]-Task_arr[i]['dead'])
            if(Task_arr[i]['size']>=4280):
                x2+=1
                Pnalti_s2+=(res[i,N]-Task_arr[i]['dead'])
            
        if(res[i,N]<=new_Task_sort[i]['dead']):
            number_MD+=1
            if(Task_arr[i]['size']<4280):
                number_MD_s1+=1
                x1+=1
#             if((Task_arr[i]['size']>=1028) and (Task_arr[i]['size']<=4280)):
#                 number_MD_s2+=1
#                 x2+=1
            if(Task_arr[i]['size']>=4280):
                number_MD_s2+=1
                x2+=1
    DSR=(number_MD*100)/Num_T
    DSR1=(number_MD_s1*100)/x1
    DSR2=(number_MD_s2*100)/x2
#     DSR3=(number_MD_s3*100)/x3
    ART=RT/Num_T
#     print('ARt',ART)
#     print('Pnalti_s1',Pnalti_s1)
#     print('Pnalti_s2',Pnalti_s2)
# #     print('Pnalti_s3',Pnalti_s3)
#     print('DSR1',DSR1)
#     print('DSR2',DSR2)
# #     print('DSR3',DSR3)
#     print('Pnalti',Pnalti)
#     print('number_MD',number_MD)
#     print('DSR',DSR)
#     print('avail',avail)
    mak=np.max(avail)
#     print('makespan:',mak)
    for j in range(all_vm):
        energy_VM[j]=((VM[j]['p_a']*(avail[j])) +(mak-(avail[j]))*VM[j]['p_i'])
        #print('energy',j,':',energy[j])
    for k in range(Num_FN+ Num_CN):
        if (k<Num_FN):
            energy_N[k]=20*mak
        if (Num_FN<=k): 
            energy_N[k]=50*mak 
    total_energy_VM=sum(energy_VM)
    total_energy_N=sum(energy_N)
    T_energy=(total_energy_VM+total_energy_N)
#     print("total_energy_VM",total_energy_VM)
#     print("total_energy_N",total_energy_N)
#     print("T_energy",T_energy)
    return mak,T_energy,Pnalti,DSR,Pnalti_s1,Pnalti_s2,DSR1,DSR2,ART

## **Main**

In [ ]:
if __name__ == "__main__":
    Num_T = int(input('Enter number of Tasks    '))
    Num_FN = int(input('Enter Number of Fog Nodes   '))
    Num_CN = int(input('Enter Number of Cloud Nodes   '))
    Pre= 40 #int(input('Enter Percentage for GRASP Algorithm   '))
    
    run=1

    ###general seting for algorithms
    max_iter=1500
    iter_GRASP=100
    N_Pop=10
    #SEED=int(input('Enter SEED  '))

    ##call dataset
    Task_arr,Task_sort=datatask(Num_T)
    # for j in range(Num_T):
    #     print("TASK",j,": ",Task_arr[j])
    new_Task_sort=sort_deadline(Task_sort)
    vmF_arr,vmC_arr,VM,all_vm,all_vmF,all_vmC = datanode(Num_FN,Num_CN)
    # for i in range(all_vm):
    #     print("VM",i,": ",VM[i])
        
    
#     for k in range(Num_T):
#         print("TASK_new",k,": ",new_Task_sort[k])
#     for j in range(Num_T):
#         print("TASK",j,": ",Task_arr[j])

#     EDf()
#     SFC()
#     GRASP(Pre)
#     algorithm3()
    
    ######result_EDF####
    sol_EDF=np.zeros((Num_T))
    makespan_EDF=np.zeros((run))
    T_energy_EDF=np.zeros((run))
    Pnalti_EDF=np.zeros((run))
    DSR_EDF=np.zeros((run))
    DSR1_EDF=np.zeros((run))
    DSR2_EDF=np.zeros((run))
    ART_EDF=np.zeros((run))
    Pnalti1_EDF=np.zeros((run))
    Pnalti2_EDF=np.zeros((run))
    ######result_SFC####
    sol_SFC=np.zeros((Num_T))
    makespan_SFC=np.zeros((run))
    T_energy_SFC=np.zeros((run))
    Pnalti_SFC=np.zeros((run))
    DSR_SFC=np.zeros((run))
    DSR1_SFC=np.zeros((run))
    DSR2_SFC=np.zeros((run))
    ART_SFC=np.zeros((run))
    Pnalti1_SFC=np.zeros((run))
    Pnalti2_SFC=np.zeros((run))
    ######result_HHOLS####
    sol_HHOLS=np.zeros((Num_T))
    makespan_HHOLS=np.zeros((run))
    T_energy_HHOLS=np.zeros((run))
    Pnalti_HHOLS=np.zeros((run))
    DSR_HHOLS=np.zeros((run))
    DSR1_HHOLS=np.zeros((run))
    DSR2_HHOLS=np.zeros((run))
    ART_HHOLS=np.zeros((run))
    Pnalti1_HHOLS=np.zeros((run))
    Pnalti2_HHOLS=np.zeros((run))
    ######result_GRASP####
    sol_GRASP=np.zeros((Num_T))
    makespan_GRASP=np.zeros((run))
    T_energy_GRASP=np.zeros((run))
    Pnalti_GRASP=np.zeros((run))
    DSR_GRASP=np.zeros((run))
    DSR1_GRASP=np.zeros((run))
    DSR2_GRASP=np.zeros((run))
    ART_GRASP=np.zeros((run))
    Pnalti1_GRASP=np.zeros((run))
    Pnalti2_GRASP=np.zeros((run))
    
    mak1=np.zeros((iter_GRASP))
    T_energy1=np.zeros((iter_GRASP))
    Pnalti1=np.zeros((iter_GRASP))
    DSR1=np.zeros((iter_GRASP))
    Pnalti_s11=np.zeros((iter_GRASP))
    Pnalti_s21=np.zeros((iter_GRASP))
    DSR11=np.zeros((iter_GRASP))
    DSR21=np.zeros((iter_GRASP))
    ART1=np.zeros((iter_GRASP))
    for i in range(run):
        nrun=i
        print('number of ran:',' ',nrun+1)
        Task_arr,Task_sort=datatask(Num_T)
        new_Task_sort=sort_deadline(Task_sort)
        vmF_arr,vmC_arr,VM,all_vm,all_vmF,all_vmC = datanode(Num_FN,Num_CN)
        makespan_EDF[i],T_energy_EDF[i],Pnalti_EDF[i],DSR_EDF[i],Pnalti1_EDF[i],Pnalti2_EDF[i],DSR1_EDF[i],DSR2_EDF[i],ART_EDF[i]=EDf()
        makespan_SFC[i],T_energy_SFC[i],Pnalti_SFC[i],DSR_SFC[i],Pnalti1_SFC[i],Pnalti2_SFC[i],DSR1_SFC[i],DSR2_SFC[i],ART_SFC[i]=SFC()
        makespan_HHOLS[i],T_energy_HHOLS[i],Pnalti_HHOLS[i],DSR_HHOLS[i],Pnalti1_HHOLS[i],Pnalti2_HHOLS[i],DSR1_HHOLS[i],DSR2_HHOLS[i],ART_HHOLS[i]=algorithm3()
        solution=0
        for k in range(iter_GRASP):
            #print('********* iteration of GRASP:',j,' ***********')
            mak1[k],T_energy1[k],Pnalti1[k],DSR1[k],Pnalti_s11[k],Pnalti_s21[k],DSR11[k],DSR21[k],ART1[k]=GRASP(Pre) 
            if (k!=0):
                if ((DSR1[solution]<DSR1[k]) or ((DSR1[k]==DSR1[solution]) and (T_energy1[solution]>T_energy1[k]) )):
                    solution=k
#                     print('solution',solution)
#                 for s in range(0,11):
#                     if(k==10*s):
#                         print('k',k)
#                         print('solution',solution)
#                         print('makespan_grasp1',mak1[solution])
#                         print('total_energy_grasp1',T_energy1[solution])
#                         print('Pnalti_grasp1',Pnalti1[solution])
#                         print('Pnalti_s1_G1',Pnalti_s11[solution])
#                         print('Pnalti_s2_G1',Pnalti_s21[solution])
#                         print('DSR_GRASP',DSR1[solution])
#                         print('DSR1_GRASP',DSR11[solution])
#                         print('DSR2_GRASP',DSR21[solution])
#                         print('ART_G1 ',ART1[solution] )
        
        makespan_GRASP[i]=mak1[solution]
        T_energy_GRASP[i]=T_energy1[solution]
        Pnalti_GRASP[i]=Pnalti1[solution]
        DSR_GRASP[i]=DSR1[solution]
        DSR1_GRASP[i]=DSR11[solution]
        DSR2_GRASP[i]=DSR21[solution]
        ART_GRASP[i]=ART1[solution]
        Pnalti1_GRASP[i]=Pnalti_s11[solution]
        Pnalti2_GRASP[i]=Pnalti_s21[solution]
    
    
    print('####Result EDF#####')
    print('makespan_EDF:',makespan_EDF,'average:',sum(makespan_EDF)/run,'max:',max(makespan_EDF)-(sum(makespan_EDF)/run),'min:',(sum(makespan_EDF)/run)-min(makespan_EDF))
    print('Total_Energy_EDF:',T_energy_EDF,'average:',sum(T_energy_EDF)/run,'max:',max(T_energy_EDF)-(sum(T_energy_EDF)/run),'min:',(sum(T_energy_EDF)/run)-min(T_energy_EDF))
    print('DSR_EDF',DSR_EDF,'average:',sum(DSR_EDF)/run,'max:',max(DSR_EDF)-(sum(DSR_EDF)/run),'min:',(sum(DSR_EDF)/run)-min(DSR_EDF))
    # print('DSR,Type1_Task_EDF',DSR1_EDF,'average:',sum(DSR1_EDF)/run,'max:',max(DSR1_EDF)-(sum(DSR1_EDF)/run),'min:',(sum(DSR1_EDF)/run)-min(DSR1_EDF))
    # print('DSR,Type2_Task_EDF',DSR2_EDF,'average:',sum(DSR2_EDF)/run,'max:',max(DSR2_EDF)-(sum(DSR2_EDF)/run),'min:',(sum(DSR2_EDF)/run)-min(DSR2_EDF))
    print('ART_EDF',ART_EDF,'average:',sum(ART_EDF)/run,'max:',max(ART_EDF)-(sum(ART_EDF)/run),'min:',(sum(ART_EDF)/run)-min(ART_EDF))
    print('####Result SFC#####')
    print('makespan_SFC:',makespan_SFC,'average:',sum(makespan_SFC)/run,'max:',max(makespan_SFC)-(sum(makespan_SFC)/run),'min:',(sum(makespan_SFC)/run)-min(makespan_SFC))
    print('Total_Energy_SFC:',T_energy_SFC,'average:',sum(T_energy_SFC)/run,'max:',max(T_energy_SFC)-(sum(T_energy_SFC)/run),'min:',(sum(T_energy_SFC)/run)-min(T_energy_SFC))
    print('DSR_SFC',DSR_SFC,'average:',sum(DSR_SFC)/run,'max:',max(DSR_SFC)-(sum(DSR_SFC)/run),'min:',(sum(DSR_SFC)/run)-min(DSR_SFC))
    # print('DSR,Type1_Task_SFC',DSR1_SFC,'average:',sum(DSR1_SFC)/run,'max:',max(DSR1_SFC)-(sum(DSR1_SFC)/run),'min:',(sum(DSR1_SFC)/run)-min(DSR1_SFC))
    # print('DSR,Type2_Task_SFC',DSR2_SFC,'average:',sum(DSR2_SFC)/run,'max:',max(DSR2_SFC)-(sum(DSR2_SFC)/run),'min:',(sum(DSR2_SFC)/run)-min(DSR2_SFC))
    print('ART_SFC',ART_SFC,'average:',sum(ART_SFC)/run,'max:',max(ART_SFC)-(sum(ART_SFC)/run),'min:',(sum(ART_SFC)/run)-min(ART_SFC))
    print('####Result HHOLS#####')
    print('makespan_HHOLS:',makespan_HHOLS,'average:',sum(makespan_HHOLS)/run,'max:',max(makespan_HHOLS)-(sum(makespan_HHOLS)/run),'min:',(sum(makespan_HHOLS)/run)-min(makespan_HHOLS))
    print('Total_Energy_HHOLS:',T_energy_HHOLS,'average:',sum(T_energy_HHOLS)/run,'max:',max(T_energy_HHOLS)-(sum(T_energy_HHOLS)/run),'min:',(sum(T_energy_HHOLS)/run)-min(T_energy_HHOLS))
    print('DSR_HHOLS',DSR_HHOLS,'average:',sum(DSR_HHOLS)/run,'max:',max(DSR_HHOLS)-(sum(DSR_HHOLS)/run),'min:',(sum(DSR_HHOLS)/run)-min(DSR_HHOLS))
    # print('DSR,Type1_Task_HHOLS',DSR1_HHOLS,'average:',sum(DSR1_HHOLS)/run,'max:',max(DSR1_HHOLS)-(sum(DSR1_HHOLS)/run),'min:',(sum(DSR1_HHOLS)/run)-min(DSR1_HHOLS))
    # print('DSR,Type2_Task_HHOLS',DSR2_HHOLS,'average:',sum(DSR2_HHOLS)/run,'max:',max(DSR2_HHOLS)-(sum(DSR2_HHOLS)/run),'min:',(sum(DSR2_HHOLS)/run)-min(DSR2_HHOLS))
    print('ART_HHOLS',ART_HHOLS,'average:',sum(ART_HHOLS)/run,'max:',max(ART_HHOLS)-(sum(ART_HHOLS)/run),'min:',(sum(ART_HHOLS)/run)-min(ART_HHOLS))
    print('####Result GRASP#####')
    print('makespan_GRASP:',makespan_GRASP,'average:',sum(makespan_GRASP)/run,'max:',max(makespan_GRASP)-(sum(makespan_GRASP)/run),'min:',(sum(makespan_GRASP)/run)-min(makespan_GRASP))
    print('Total_Energy_GRASP:',T_energy_GRASP,'average:',sum(T_energy_GRASP)/run,'max:',max(T_energy_GRASP)-(sum(T_energy_GRASP)/run),'min:',(sum(T_energy_GRASP)/run)-min(T_energy_GRASP))
    print('DSR_GRASP',DSR_GRASP,'average:',sum(DSR_GRASP)/run,'max:',max(DSR_GRASP)-(sum(DSR_GRASP)/run),'min:',(sum(DSR_GRASP)/run)-min(DSR_GRASP))
    # print('DSR,Type1_Task_GRASP',DSR1_GRASP,'average:',sum(DSR1_GRASP)/run,'max:',max(DSR1_GRASP)-(sum(DSR1_GRASP)/run),'min:',(sum(DSR1_GRASP)/run)-min(DSR1_GRASP))
    # print('DSR,Type2_Task_GRASP',DSR2_GRASP,'average:',sum(DSR2_GRASP)/run,'max:',max(DSR2_GRASP)-(sum(DSR2_GRASP)/run),'min:',(sum(DSR2_GRASP)/run)-min(DSR2_GRASP))
    print('ART_GRASP',ART_GRASP,'average:',sum(ART_GRASP)/run,'max:',max(ART_GRASP)-(sum(ART_GRASP)/run),'min:',(sum(ART_GRASP)/run)-min(ART_GRASP))


Enter number of Tasks    100
Enter Number of Fog Nodes   24
Enter Number of Cloud Nodes   4
number of ran:   1
####Result EDF#####
makespan_EDF: [35.78175313] average: 35.78175313059034 max: 0.0 min: 0.0
Total_Energy_EDF: [155262.27728496] average: 155262.27728495523 max: 0.0 min: 0.0
DSR_EDF [63.] average: 63.0 max: 0.0 min: 0.0
ART_EDF [3.54260686] average: 3.5426068595728917 max: 0.0 min: 0.0
####Result SFC#####
makespan_SFC: [3.17025114] average: 3.1702511447204107 max: 0.0 min: 0.0
Total_Energy_SFC: [17114.89293876] average: 17114.892938761885 max: 0.0 min: 0.0
DSR_SFC [85.] average: 85.0 max: 0.0 min: 0.0
ART_SFC [1.66010601] average: 1.6601060135242434 max: 0.0 min: 0.0
####Result HHOLS#####
makespan_HHOLS: [4.49446064] average: 4.494460641399417 max: 0.0 min: 0.0
Total_Energy_HHOLS: [21883.80878628] average: 21883.808786280035 max: 0.0 min: 0.0
DSR_HHOLS [67.] average: 67.0 max: 0.0 min: 0.0
ART_HHOLS [2.32221377] average: 2.322213765575456 max: 0.0 min: 0.0
####Result GRASP###